## Imports 

In [ ]:
import argparse
import easydict
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
from plotread import *
import shap
from collections import Counter
from alibi.explainers.pd_variance import PartialDependenceVariance, plot_pd_variance
from alibi.explainers import PartialDependence, plot_pd
from alibi.explainers import ALE
from alibi.explainers.ale import plot_ale
from alibi.explainers import PermutationImportance
%matplotlib inline
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# init the JS visualization code
shap.initjs()

In [ ]:
# import tensorflow._api.v2.compat.v1 as tf                 # does not work for python=3.7
from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()                          # does not work for thesis, except with DeepExplainer
tf.compat.v1.disable_eager_execution()                    # does not work

# from tensorflow.python.ops.numpy_ops import np_config     # does not work for python=3.7
# np_config.enable_numpy_behavior()                         # does not work for python=3.7

# from tensorflow.math import reduce_prod                 # does not work
# from tensorflow.python.ops.math_ops import reduce_prod  # does not work

# sess = tf.compat.v1.keras.backend.get_session()                   # does not work for thesis
# sess.run(tf.compat.v1.global_variables_initializer())             # does not work for thesis
# writer = tf.compat.v1.summary.FileWriter("c:\\tmp", sess.graph)   # does not work for thesis
# writer.close()     

## Functions 

In [ ]:
def custom_predictor(X):
    print("X:\n")
    print(X)
    print(X.shape)
    
    predictions = model.predict(X)
    print("predictions:\n")
    print(predictions)
    print(predictions.shape)
    
    predictions_reshaped = np.reshape(predictions, (1, 4000), order='F')
    print("predictions_reshaped:\n")
    print(predictions_reshaped)
    print(predictions_reshaped.shape)

    return predictions_reshaped

In [ ]:
def main():
    ###########################################################################
    # Parser Definition
    ###########################################################################
    opt = easydict.EasyDict({
        "model": "GRU",
        "datapath": "Dataset1/",
        "savepath": "Experiment1/GRU/8/Run1/",
        "extension": ".dat",
        "batch_size": 32,
        "plots_in": False,
        "plots_out": True
    })

    ###########################################################################
    # Variables Definition
    ###########################################################################
    nin = ['time', 'PLML2', 'PLMR', 'AVBL', 'AVBR']
    nout = ['time', 'DB1', 'LUAL', 'PVR', 'VB1']
    neurons = ['time','DB1','LUAL','PVR','VB1','PLML2','PLMR','AVBL','AVBR']

    ###########################################################################
    # Read data
    ###########################################################################
    files = getdata(opt.datapath, opt.extension)
    train, valid, test = splitdata(files)
    trainx, trainy = readdata(opt.datapath, train, neurons, nin, nout)
    validx, validy = readdata(opt.datapath, valid, neurons, nin, nout)
    testx, testy = readdata(opt.datapath, test, neurons, nin, nout)
    if opt.plots_in:
        plotdata(trainx, '/train_data', '/x', opt.model, opt.savepath)
        plotdata(trainy, '/train_data', '/y', opt.model, opt.savepath)
        plotdata(validx, '/valid_data', '/x', opt.model, opt.savepath)
        plotdata(validy, '/valid_data', '/y', opt.model, opt.savepath)
        plotdata(testx, '/test_data', '/x', opt.model, opt.savepath)
        plotdata(testy, '/test_data', '/y', opt.model, opt.savepath)
        
    ###########################################################################
    # Load Model and Evaluate
    ###########################################################################
    output_size = 4
    global model
    model = load_model(opt.savepath + 'model.h5')
    model.summary()

    print("Starting to explain...")
    X_train = np.array(trainx)
    X_test = np.array(testx)

    nin_names = nin[1:len(nin)]

    return X_train, X_test, nin_names

## Code 

In [ ]:
X_train, X_test, nin_names = main()

In [ ]:
background = X_train[np.random.choice(X_train.shape[0], 1, replace=False)]
print("background.shape=")
print(background.shape)

In [ ]:
train_x_reshaped = np.reshape(X_train, (20, 4000), order='F')
print("train_x_reshaped.shape=")
print(train_x_reshaped.shape)

In [ ]:
test_x_reshaped = np.reshape(X_test, (10, 4000), order='F')
print("test_x_reshaped.shape=")
print(test_x_reshaped.shape)

#### Explainer

##### ALE

In [ ]:
# ale = ALE(model.predict, feature_names=nin_names)
ale = ALE(custom_predictor, feature_names=nin_names)

In [ ]:
# exp = ale.explain(X_test)
exp = ale.explain(test_x_reshaped)

##### PD

In [ ]:
# define explainer
explainer = PartialDependenceVariance(predictor=model.predict,
                                      feature_names=nin_names,
                                      verbose=True)

In [ ]:
# define explainer
explainer = PartialDependenceVariance(predictor=custom_predictor,
                                      feature_names=nin_names,
                                      verbose=True)

In [ ]:
exp_importance = explainer.explain(X=X_train[0],
                                   method='interaction')

##### PD ICE

In [ ]:
explainer = PartialDependence(predictor=custom_predictor,
                              feature_names=nin_names)

In [ ]:
explainer = PartialDependence(predictor=model.predict,
                              feature_names=nin_names)

In [ ]:
# select temperature, humidity, wind speed, and season
features = [nin_names.index('PLML2'), 
            nin_names.index('PLMR'), 
            nin_names.index('AVBL'),
            nin_names.index('AVBR')]

In [ ]:
# compute explanations
exp = explainer.explain(X=X_train[0],
                        features=features,
                        kind='average')

In [ ]:
# compute explanations
exp = explainer.explain(X=train_x_reshaped,
                        features=features,
                        kind='average')

##### PDP

In [ ]:
shap.plots.partial_dependence(0, model.predict, background, feature_names=nin_names,
                              ice=False, model_expected_value=True, feature_expected_value=True)

In [ ]:
shap.plots.partial_dependence(1, model.predict, background, feature_names=nin_names,
                              ice=False, model_expected_value=True, feature_expected_value=True)

In [ ]:
shap.plots.partial_dependence(2, model.predict, background, feature_names=nin_names,
                              ice=False, model_expected_value=True, feature_expected_value=True)

In [ ]:
shap.plots.partial_dependence(3, model.predict, background, feature_names=nin_names,
                              ice=False, model_expected_value=True, feature_expected_value=True)

In [ ]:
shap.plots.partial_dependence(999, model.predict, background, hist=True,
                              ice=False, model_expected_value=True, feature_expected_value=True)